In [1]:
import epxexec # Epistemix package for running FRED simulations within Python
import epxexec.epxresults as epxresults # Epistemix package for managing and accessing FRED simulation outputs
from epxexec.fred_job import fred_job
import pandas as pd

import os
from pathlib import Path
os.environ["FRED_PROJECT"] = str(Path.cwd())

# Ground logistics model

This model represents a ground transportation network. Cities represent nodes in the network and roads are edges. A table of trucks agents is read in at the beginning of the simulation. There are no human agents in the model. 

## Model components

- `city_locations.fred`: contains the GENERATE\_LOCATIONS condition that creates a network of cities
- `trucks.fred`: contains the GENERATE\_TRUCKS condition that reads the trucks agent file and the MOVE\_TRUCKS condition that picks next locations for trucks to travel to
- `travel_disruption.fred`: contains the DISRUPTION condition that creates travel disruptions at certain nodes as defined by an end user

In [8]:
ground_logistics = fred_job(program="model/main.fred")


fred_job: starting job  at Tue Oct 31 11:41:13 PM UTC 2023

fred_compile -p /home/epx/epistemix-community-library/Ground-Shipping-Logistics/model/main.fred
No errors found.
No warnings.

fred_job: running job 117 117 ...
FRED starting at Tue Oct 31 11:41:14 PM UTC 2023

FRED finished at Tue Oct 31 11:41:16 PM UTC 2023

fred_job: finished job 117 117 at Tue Oct 31 11:41:19 PM UTC 2023




## Data outputs

Let's take a look at the output `travel_info.csv`. The data summarizes the progress of each truck on the journey from its most recently visited city to its next destination by recording the remaining distance left to travel on each day of the simulation.

In [10]:
travel_log_df = ground_logistics.runs[1].get_csv_output('travel_log.csv')
travel_log_df

,ID,DAY,HOME_CITY,MOST_RECENT_CITY,NEXT_CITY,DISTANCE_REMAINING
0,1000000000,20200102,900000001,900000001,900000010,2437
1,1000000001,20200102,900000001,900000001,900000009,1048
2,1000000002,20200102,900000001,900000001,900000005,1930
3,1000000003,20200102,900000001,900000001,900000008,2323
4,1000000004,20200102,900000001,900000001,900000010,2437
...,...,...,...,...,...,...
983,1000000035,20200201,900000008,900000003,900000008,0
984,1000000037,20200201,900000009,900000009,900000010,0
985,1000000041,20200201,900000009,900000002,900000001,2293
986,1000000042,20200201,900000009,900000004,900000001,0
